In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

import os 

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

from IPython.display import SVG, Image
# from livelossplot import PlotLossesKeras
# from livelossplot.keras import PlotLossesCallback
from livelossplot import PlotLossesKeras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
# for expression in os.listdir('train/'):
#     if expression != '.DS_Store':
#         print(str(len(os.listdir('train/' + expression)))+ ' ' + expression)
#     else:
#         continue


img_size = 48
batch_size = 64

datagen_train = ImageDataGenerator(horizontal_flip=True)
train_generator = datagen_train.flow_from_directory('train/',
                                                    target_size = (img_size, img_size),
                                                    color_mode = 'grayscale',
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical',
                                                    shuffle = True)

datagen_validation = ImageDataGenerator(horizontal_flip=True)
validation_generator = datagen_train.flow_from_directory('test/',
                                                    target_size = (img_size, img_size),
                                                    color_mode = 'grayscale',
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical')

In [5]:
## Create CNN Model:

model = Sequential()


# 1st conv
model.add(Conv2D(64, (3,3), padding = 'same', input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# 2
model.add(Conv2D(128, (5,5), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# 3
model.add(Conv2D(512, (3,3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#4
model.add(Conv2D(512, (3,3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# flatten

model.add(Flatten())

model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))

optimizer = Adam(lr = 0.0005)

model.compile(optimizer=optimizer,
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Found 28708 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization_6 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
activation_6 (Activation)    (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 128)       204928    
___________________________________

In [9]:
# Train and Evaluate Model

epochs = 15
steps_per_epoch = train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size

checkpoint = ModelCheckpoint('model_weights.h5',
                              monitor = 'val_accuracy',
                              save_weights_only = True,
                              mode = 'max',
                              verbose= 1)

reduce_lr = ReduceLROnPlateau(monitor= 'val_loss',
                              factor= 0.1,
                              patience=2,
                              min_lr= 0.00001, 
                              model = 'auto')

callbacks = [checkpoint, reduce_lr]

history = model.fit(x = train_generator, 
                    steps_per_epoch = steps_per_epoch, 
                    epochs = epochs, 
                    validation_data= validation_generator, 
                    validation_steps = validation_steps, 
                    callbacks = callbacks,
                   workers = -1)

Epoch 1/15
448/448 [==============================] - ETA: 0s - loss: 1.7741 - accuracy: 0.3189
Epoch 00001: saving model to model_weights.h5
448/448 [==============================] - 461s 1s/step - loss: 1.7741 - accuracy: 0.3189 - val_loss: 1.6590 - val_accuracy: 0.3857 - lr: 5.0000e-04
Epoch 2/15
448/448 [==============================] - ETA: 0s - loss: 1.4631 - accuracy: 0.4381
Epoch 00002: saving model to model_weights.h5
448/448 [==============================] - 375s 838ms/step - loss: 1.4631 - accuracy: 0.4381 - val_loss: 1.3313 - val_accuracy: 0.4849 - lr: 5.0000e-04
Epoch 3/15
448/448 [==============================] - ETA: 0s - loss: 1.3302 - accuracy: 0.4907
Epoch 00003: saving model to model_weights.h5
448/448 [==============================] - 375s 836ms/step - loss: 1.3302 - accuracy: 0.4907 - val_loss: 1.2796 - val_accuracy: 0.5078 - lr: 5.0000e-04
Epoch 4/15
448/448 [==============================] - ETA: 0s - loss: 1.2494 - accuracy: 0.5208
Epoch 00004: saving model

In [12]:
## Represent Model in JSON String:
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)